# Simulación de Taxis de NYC por el método Montecarlo

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import timedelta

Cargando datos

In [4]:
taxi = pd.read_parquet('../../Data/Data_taxi_sim.parquet')
fuel = pd.read_parquet('../../Data/Eda_fuel_vehicles.parquet')

Transformaciones

In [5]:
taxi['time'] = taxi['tpep_dropoff_datetime'] - taxi['tpep_pickup_datetime']
taxi['time'] = taxi['time'].apply(lambda x: int(x.total_seconds() // 60))
taxi = taxi.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'], axis=1)

### Funciones

In [2]:
def rango(df,column:str):
    '''
Esta función devuelve un rango de valores que excluye outliers, la función recibe
    - df - Un dataFrame
    - column - Una columna en str
    La función devuelve una tupla con el intervalo valido
    '''
    statistics = df[column].describe()
    IQR = statistics['75%'] - statistics['25%']
    RI = statistics['25%'] - 1.5 * IQR       # RAngo inferior
    if RI < 0:
        RI = 0
    RS = statistics['75%'] + 1.5 * IQR       # Rango superior
    if RI == RS:
        return (RI-1,RS)
    else:
        return (RI , RS)

In [90]:
def trip(UpLocationId:int,Hour):
    '''
    Esta función devuelve caracteristicas de la ruta entre dos puntos. La función recibe:
        - UpLocationId. Id de la zona de inicio del viaje
        - Hour. Hora en la que inicia el viaje en formato `timedelta`
        
    La función devuelve un diccionario con:
        - route. Ruta
        - UpLocationId. Id zona de inicio del viaje
        - DOLocationID. Id zona final del viaje
        - Hour_i. Hora de inicio del viaje
        - Hour_e. Hora en la que finaliza el viaje
        - trip_time. Tiempo del viaje en minutos
        - trip_distance. Distancia recorrida
        - total_amount. Costo del viaje
    '''
    filtro = taxi[taxi['PULocationID'] == UpLocationId]             # Filtramos registros de la zona de inicio del viaje
    destino = np.random.choice(filtro['DOLocationID'])              # Definimos zona de destino del viaje
    filtro = filtro[filtro['DOLocationID'] == destino]              # Actualizamos el filtro
    # Determinar el rango de valores que excluyen outliers
    rango_tiempo = rango(filtro,'time')
    rango_distancia = rango(filtro,'trip_distance')
    # Eliminando registros con datos atipicos
    if len(filtro) != 1:
        filtro = filtro[(filtro['time'] >= rango_tiempo[0]) & (filtro['time'] <= rango_tiempo[1]) & 
                    (filtro['trip_distance'] >= rango_distancia[0]) &
                      (filtro['trip_distance'] <= rango_distancia[1])]
        route = np.random.choice(filtro['route'])
    else:
        filtro = filtro.reset_index()
        route = filtro['route'].iloc[0]
    # Extrayendo datos
    
    distance = np.random.choice(filtro['trip_distance'])
    time = filtro[filtro['trip_distance'] == distance]['time'].mean()
    amount = filtro[filtro['trip_distance'] == distance]['total_amount'].mean()
    Hour_e = Hour + timedelta(hours=0, minutes = time)
    return {"route":route,"UpLocationId":UpLocationId,"DOLocationID":destino,"Hour_i":Hour,"Hour_e":Hour_e,
            "trip_time":time,"trip_distance":distance,"total_amount":amount}

In [64]:
def simulacion(n:int,hora_inicio,hora_fin):
    '''
    Función simulacion devuelve un Df con las iteracciones solicitadas por el usuario en el rango de tiempo solicitado.
    - n. Número de simlaciones Montecarlo en valor entero
    - hora_inicio. Hora en la que se inicia la simulación en formato `timedelta`
    - hora_fin. Hora en la que termina la simulación en formato `timedelta`

    Devuelve lista con datos generados por cada iteración y resumen de cada iteración
    
    La lista con datos contiene:
    - route. Ruta
    - UpLocationId. Id zona de inicio del viaje
    - DOLocationID. Id zona final del viaje
    - Hour_i. Hora de inicio del viaje
    - Hour_e. Hora en la que finaliza el viaje
    - trip_time. Tiempo del viaje en minutos
    - trip_distance. Distancia recorrida
    - total_amount. Costo del viaje

    El resumen de las iteraciones contiene la siguiente información:
    - interacion. Numero de iteracion
    - trip_time. El tiempo total de la iteración
    - trip_time_mean. Tiempo promedio por viaje
    - trip_distance. Distancia total recorrida para la iteración
    - trip_mean. Distancia promedio por viaje
    - total_amount. Ingreso total para la iteración
    - amount_mean. Ingreso promedio por viaje
    '''
    lista = []
    resultados = []
    no = 0
    for i in range(0,n):
        hr_ini = hora_inicio
        idzonaInicio = np.random.choice(taxi['PULocationID'])
        data = []
        while hr_ini <= hora_fin:
            a = trip(idzonaInicio,hr_ini)
            data.append(a)
            idzonaInicio = a['DOLocationID']
            hr_ini = a['Hour_e']
        df = pd.DataFrame(data)
        lista.append(df)
    for j in lista:
        res = {"interacion":no,"trip_time":timedelta(minutes=j['trip_time'].sum()),
            "trip_time_mean":timedelta(minutes=j['trip_time'].mean()),
            "trip_distance":j['trip_distance'].sum(),"trip_mean":j['trip_distance'].mean(),
            "total_amount":j['total_amount'].sum(),"amount_mean":j['total_amount'].mean()}
        resultados.append(res)
        no += 1
    df = pd.DataFrame(resultados)
    return lista, df

In [65]:
inicio = timedelta(hours=5,minutes=0)
fin = timedelta(hours=18,minutes=0)
iteraciones = 5
a = simulacion(iteraciones,inicio,fin)

### Simulaciones

Simulación de un solo viaje con zona de inicio aleatorio

In [67]:
inicio = np.random.choice(taxi['PULocationID'])
trip(inicio,timedelta(hours=2,minutes=4))

{'route': 'Central Park -- Upper West Side South',
 'UpLocationId': 43,
 'DOLocationID': 239,
 'Hour_i': datetime.timedelta(seconds=7440),
 'Hour_e': datetime.timedelta(seconds=7756, microseconds=483516),
 'trip_time': 5.274725274725275,
 'trip_distance': 0.9,
 'total_amount': 14.260549450549451}

Simulación de una unica corrida en un horario determinado

In [79]:
inicio = timedelta(hours=5,minutes=0)
fin = timedelta(hours=18,minutes=0)
iteraciones = 1
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteración nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas")

La simulacion con 1 iteración nos brinda:
El recorrido simulado es de 155.48000000000002 millas 
El recorrido promedio por viaje es de 2.99 millas 
El ingreso simulado es $2295.3948032428852 
El ingreso promedio por viaje es de $44.14220775467087 
Cada viaje duro en promedio 0 days 00:15:06.255846 
Para un total de 0 days 13:05:25.304007 horas


Simulación con dos iteraciones

In [80]:
inicio = timedelta(hours=5,minutes=0)
fin = timedelta(hours=18,minutes=0)
iteraciones = 2
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas")

La simulacion con 2 iteraciones nos brinda:
El recorrido simulado es de 168.265 millas 
El recorrido promedio por viaje es de 3.298284807764091 millas 
El ingreso simulado es $1281.6523969314514 
El ingreso promedio por viaje es de $25.00716375783923 
Cada viaje duro en promedio 0 days 00:15:31.348001500 
Para un total de 0 days 13:18:40.527943500 horas


Simulación con 10 iteraciones

In [81]:
inicio = timedelta(hours=5,minutes=0)
fin = timedelta(hours=18,minutes=0)
iteraciones = 10
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas")

La simulacion con 10 iteraciones nos brinda:
El recorrido simulado es de 156.05900000000003 millas 
El recorrido promedio por viaje es de 3.2271534627100196 millas 
El ingreso simulado es $1326.1508954090125 
El ingreso promedio por viaje es de $26.80227470873179 
Cada viaje duro en promedio 0 days 00:16:34.305927500 
Para un total de 0 days 13:18:58.212558800 horas


Simulación con 20 iteraciones

In [82]:
inicio = timedelta(hours=5,minutes=0)
fin = timedelta(hours=18,minutes=0)
iteraciones = 20
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas")

La simulacion con 20 iteraciones nos brinda:
El recorrido simulado es de 157.932 millas 
El recorrido promedio por viaje es de 2.915322673448943 millas 
El ingreso simulado es $1714.076541216303 
El ingreso promedio por viaje es de $29.878696581750354 
Cada viaje duro en promedio 0 days 00:14:30.428213700 
Para un total de 0 days 13:09:00.177295500 horas


Simulación con 50 iteraciones

In [83]:
inicio = timedelta(hours=5,minutes=0)
fin = timedelta(hours=18,minutes=0)
iteraciones = 50
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas")

La simulacion con 50 iteraciones nos brinda:
El recorrido simulado es de 157.2662 millas 
El recorrido promedio por viaje es de 2.8396358946470492 millas 
El ingreso simulado es $1548.5351828958803 
El ingreso promedio por viaje es de $26.836452375843646 
Cada viaje duro en promedio 0 days 00:14:19.712644220 
Para un total de 0 days 13:15:05.621734020 horas


Simulación con 100 iteraciones

In [85]:
inicio = timedelta(hours=5,minutes=0)
fin = timedelta(hours=18,minutes=0)
iteraciones = 100
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas")

La simulacion con 100 iteraciones nos brinda:
El recorrido simulado es de 156.5203 millas 
El recorrido promedio por viaje es de 2.9112676872666263 millas 
El ingreso simulado es $1564.2301132800924 
El ingreso promedio por viaje es de $27.479227678946575 
Cada viaje duro en promedio 0 days 00:14:41.006806620 
Para un total de 0 days 13:12:25.481095980 horas


Simulación con 200 iteraciones

In [91]:
inicio = timedelta(hours=5,minutes=0)
fin = timedelta(hours=18,minutes=0)
iteraciones = 200
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas")

La simulacion con 200 iteraciones nos brinda:
El recorrido simulado es de 157.40884999999997 millas 
El recorrido promedio por viaje es de 2.915201707421895 millas 
El ingreso simulado es $1596.0833989268112 
El ingreso promedio por viaje es de $27.944189685579047 
Cada viaje duro en promedio 0 days 00:14:39.787645925 
Para un total de 0 days 13:13:56.219479410 horas


Simulación con 500 iteraciones

In [104]:
inicio = timedelta(hours=5,minutes=0)
fin = timedelta(hours=18,minutes=0)
iteraciones = 500
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas")

La simulacion con 500 iteraciones nos brinda:
El recorrido simulado es de 156.84672 millas 
El recorrido promedio por viaje es de 2.8882186165953088 millas 
El ingreso simulado es $1783.3341614144017 
El ingreso promedio por viaje es de $28.828078095428584 
Cada viaje duro en promedio 0 days 00:14:38.254037030 
Para un total de 0 days 13:14:06.401324488 horas


Simulación con 1000 iteraciones

In [105]:
inicio = timedelta(hours=5,minutes=0)
fin = timedelta(hours=18,minutes=0)
iteraciones = 1000
a = simulacion(iteraciones,inicio,fin)
print(f"La simulacion con {len(a[1])} iteraciones nos brinda:\nEl recorrido simulado es de {a[1]['trip_distance'].mean()} millas \nEl recorrido promedio por viaje es de {a[1]['trip_mean'].mean()} millas \nEl ingreso simulado es ${a[1]['total_amount'].mean()} \nEl ingreso promedio por viaje es de ${a[1]['amount_mean'].mean()} \nCada viaje duro en promedio {a[1]['trip_time_mean'].mean()} \nPara un total de {a[1]['trip_time'].mean()} horas")

La simulacion con 1000 iteraciones nos brinda:
El recorrido simulado es de 206.59527000000003 millas 
El recorrido promedio por viaje es de 3.7342421019383867 millas 
El ingreso simulado es $1688.0670651946282 
El ingreso promedio por viaje es de $28.30094430597562 
Cada viaje duro en promedio 0 days 00:14:24.864502248 
Para un total de 0 days 13:14:49.961863276 horas


Dataframe con el resumen de las 1000 iteraciones

In [121]:
a[1]

,interacion,trip_time,trip_time_mean,trip_distance,trip_mean,total_amount,amount_mean
0,0,0 days 13:08:24.635731,0 days 00:13:49.905890,169.37,2.971404,1998.053453,35.053569
1,1,0 days 13:11:10.137147,0 days 00:11:27.973002,135.53,1.964203,1462.274730,21.192387
2,2,0 days 13:10:22.092379,0 days 00:16:27.960258,158.64,3.305000,1389.489095,28.947689
3,3,0 days 13:03:55.714569,0 days 00:15:04.532972,141.75,2.725962,1350.193030,25.965251
4,4,0 days 13:09:52.646489,0 days 00:16:07.196867,187.42,3.824898,1289.348514,26.313235
...,...,...,...,...,...,...,...
995,995,0 days 13:00:36.882742,0 days 00:12:35.433593,179.06,2.888065,1501.436633,24.216720
996,996,0 days 13:24:07.103452,0 days 00:13:51.846611,145.66,2.511379,1335.804127,23.031106
997,997,0 days 13:02:10.882735,0 days 00:12:13.295043,162.53,2.539531,1455.316371,22.739318
998,998,0 days 13:00:30.537232,0 days 00:16:36.394409,166.55,3.543617,1333.937074,28.381640


Dataframe con el registro de los viajes de una sola iteración

In [120]:
a[0][5]

,route,UpLocationId,DOLocationID,Hour_i,Hour_e,trip_time,trip_distance,total_amount
0,Midtown Center -- Times Sq/Theatre District,161,230,0 days 05:00:00,0 days 05:11:22.758621,11.379310,0.97,17.871897
1,Times Sq/Theatre District -- Penn Station/Madi...,230,186,0 days 05:11:22.758621,0 days 05:21:50.866729,10.468468,1.13,17.152793
2,Penn Station/Madison Sq West -- Union Sq,186,234,0 days 05:21:50.866729,0 days 05:28:31.158700,6.671533,0.92,15.123285
3,Union Sq -- Midtown South,234,164,0 days 05:28:31.158700,0 days 05:35:11.369226,6.670175,0.70,14.663193
4,Midtown South -- Flatiron,164,90,0 days 05:35:11.369226,0 days 05:42:43.133932,7.529412,1.18,16.027647
...,...,...,...,...,...,...,...,...
59,East Harlem North -- Upper East Side South,74,237,0 days 16:55:21.603297,0 days 17:11:21.603297,16.000000,2.70,23.935000
60,Upper East Side South -- Upper East Side South,237,237,0 days 17:11:21.603297,0 days 17:16:58.325230,5.612032,0.70,13.847101
61,Upper East Side South -- Manhattan Valley,237,151,0 days 17:16:58.325230,0 days 17:29:29.325230,12.516667,2.40,22.931167
62,Manhattan Valley -- Midtown Center,151,161,0 days 17:29:29.325230,0 days 17:44:29.325230,15.000000,2.62,20.300000
